<a href="https://colab.research.google.com/github/taylor33189-beep/Taylor_Hoskins_Repository/blob/main/Multi_Way_ANOVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Data Loading and Preparation


df = pd.read_csv('/lowbwt.csv')

# Display the first few rows of the dataframe to understand its structure
display(df.head())

# Calculate the median values for gestational age and apgar5 score.
# These medians will be used to create binary categorical variables.
gestage_median = df['gestage'].median()
apgar5_median = df['apgar5'].median()

# Create derived binary variables based on the medians.
# 'gestage_low': 1 if gestational age is below the median, 0 otherwise.
df['gestage_low'] = np.where(df['gestage'] < gestage_median, 1, 0)
# 'apgar5_low': 1 if apgar5 score is below the median, 0 otherwise.
df['apgar5_low'] = np.where(df['apgar5'] < apgar5_median, 1, 0)

# Convert the derived binary variables to the 'category' data type.
# This is important for proper handling in statistical models like ANOVA.
df['gestage_low'] = df['gestage_low'].astype('category')
df['apgar5_low'] = df['apgar5_low'].astype('category')

# Display the dataframe with the new categorical variables
display(df.head())


# 2. Side-by-Side Box Plots

# Create a figure to hold the box plots.
plt.figure(figsize=(12, 6))

# Create the first subplot for systolic blood pressure (sbp) by gestational age category.
plt.subplot(1, 2, 1)
# Generate the box plot. 'column' is the variable to plot, 'by' is the grouping variable.
df.boxplot(column='sbp', by='gestage_low', ax=plt.gca())
# Set the title and labels for the first box plot.
plt.title('Systolic Blood Pressure (sbp) by Gestational Age (Below/Above Median)')
plt.suptitle('') # Remove the automatic suptitle generated by boxplot
plt.xlabel('Gestational Age (1: Below Median, 0: Above or Equal to Median)')
plt.ylabel('Systolic Blood Pressure (sbp)')

# Create the second subplot for systolic blood pressure (sbp) by apgar5 score category.
plt.subplot(1, 2, 2)
# Generate the box plot.
df.boxplot(column='sbp', by='apgar5_low', ax=plt.gca())
# Set the title and labels for the second box plot.
plt.title('Systolic Blood Pressure (sbp) by Apgar5 Score (Below/Above Median)')
plt.suptitle('')
plt.xlabel('Apgar5 Score (1: Below Median, 0: Above or Equal to Median)')
plt.ylabel('Systolic Blood Pressure (sbp)')

# Adjust the layout to prevent titles and labels from overlapping.
plt.tight_layout()

boxplot_path = 'sbp_boxplots.png'
plt.savefig(boxplot_path)

plt.close()


# 3. Two-way ANOVA Model Selection

# Define the formula for a two-way ANOVA model including an interaction term.
# C() is used to treat the variables as categorical.
# The * symbol indicates that both main effects and their interaction should be included.
formula_interaction = 'sbp ~ C(gestage_low) * C(apgar5_low)'

# Fit the model with interaction using Ordinary Least Squares (OLS).
model_interaction = smf.ols(formula_interaction, data=df).fit()

# Perform ANOVA to assess the significance of the model terms.
# typ=2 specifies Type II ANOVA, which is appropriate for unbalanced designs.
anova_interaction = anova_lm(model_interaction, typ=2)

# Extract the p-value for the interaction term.
# A small p-value (typically < 0.05) indicates a significant interaction.
p_interaction = anova_interaction.loc['C(gestage_low):C(apgar5_low)', 'PR(>F)']

# Determine the optimal model based on the interaction term's p-value.
# If the interaction is significant, use the model with interaction.
# Otherwise, use a simpler model without interaction (main effects only).
if p_interaction < 0.05:
    optimal_model = model_interaction
    formula_optimal = formula_interaction
    model_type = "with interaction"
else:
    # Define the formula for a two-way ANOVA model without an interaction term.
    # The + symbol includes only the main effects.
    formula_no_interaction = 'sbp ~ C(gestage_low) + C(apgar5_low)'
    # Fit the model without interaction.
    model_no_interaction = smf.ols(formula_no_interaction, data=df).fit()
    optimal_model = model_no_interaction
    formula_optimal = formula_no_interaction
    model_type = "without interaction"

# Print information about the chosen optimal model.
print(f"Optimal Model Chosen: {model_type} ({formula_optimal})")
# Print the theoretical coefficients from the fitted optimal model.
print("\nOptimal Model Coefficients (Theoretical):")
print(optimal_model.params)


# 4. Permutation Testing for Factors in the Optimal Model (B=1000)

# Define the number of permutations to perform.
B = 1000

# Re-fit the optimal model (in this case, the no-interaction model) to get the observed F-statistics.
# We do this specifically for the permutation test base.
model_for_permutation = smf.ols(formula_optimal, data=df).fit()
anova_for_permutation = anova_lm(model_for_permutation, typ=2)

# Get the observed F-statistics for each factor in the optimal model.
# These are the F-statistics from the original data.
f_obs_gestage = anova_for_permutation.loc['C(gestage_low)', 'F']
f_obs_apgar5 = anova_for_permutation.loc['C(apgar5_low)', 'F']

# Initialize lists to store F-statistics from permutation tests.
f_perm_gestage = []
f_perm_apgar5 = []

# Perform the permutation tests.
# We iterate B times to create B permuted datasets.
for _ in range(B):
    # Permutation for gestage_low: Shuffle the 'gestage_low' column randomly.
    # This breaks any relationship between 'gestage_low' and 'sbp' while keeping other relationships intact.
    df_perm_gestage = df.copy()
    df_perm_gestage['gestage_low'] = np.random.permutation(df['gestage_low'])
    # Fit the optimal model to the permuted data.
    model_perm_gestage = smf.ols(formula_optimal, data=df_perm_gestage).fit()
    # Perform ANOVA on the permuted data and store the F-statistic for gestage_low.
    anova_perm_gestage = anova_lm(model_perm_gestage, typ=2)
    f_perm_gestage.append(anova_perm_gestage.loc['C(gestage_low)', 'F'])

    # Permutation for apgar5_low: Shuffle the 'apgar5_low' column randomly.
    # This breaks any relationship between 'apgar5_low' and 'sbp'.
    df_perm_apgar5 = df.copy()
    df_perm_apgar5['apgar5_low'] = np.random.permutation(df['apgar5_low'])
    # Fit the optimal model to the permuted data.
    model_perm_apgar5 = smf.ols(formula_optimal, data=df_perm_apgar5).fit()
    # Perform ANOVA on the permuted data and store the F-statistic for apgar5_low.
    anova_perm_apgar5 = anova_lm(model_perm_apgar5, typ=2)
    f_perm_apgar5.append(anova_perm_apgar5.loc['C(apgar5_low)', 'F'])

# Calculate empirical p-values from the permutation tests.
# The p-value is the proportion of permutation F-statistics that are equal to or greater than the observed F-statistic.
p_value_gestage_perm = (np.array(f_perm_gestage) >= f_obs_gestage).mean()
p_value_apgar5_perm = (np.array(f_perm_apgar5) >= f_obs_apgar5).mean()

# Print the results of the permutation tests.
print("\nPermutation Test p-values:")
print(f"Gestage_low p-value: {p_value_gestage_perm:.4f}")
print(f"Apgar5_low p-value: {p_value_apgar5_perm:.4f}")


# 5. Bootstrap Estimation of Coefficients for the Optimal Model (B=1000)

# Initialize a list to store the coefficients from each bootstrap sample.
bootstrap_coefficients = []

# Perform bootstrap resampling.
# Iterate B times to create B bootstrap samples.
for _ in range(B):
    # Create a bootstrap sample by randomly sampling rows from the original dataframe with replacement.
    bootstrap_sample = df.sample(n=len(df), replace=True)

    # Fit the optimal model to the bootstrap sample.
    model_boot = smf.ols(formula_optimal, data=bootstrap_sample).fit()

    # Store the coefficients from the fitted model.
    bootstrap_coefficients.append(model_boot.params)

# Convert the list of coefficient Series into a DataFrame for easier analysis.
boot_results_df = pd.DataFrame(bootstrap_coefficients)

# Calculate the mean of the coefficients across all bootstrap samples.
boot_means = boot_results_df.mean()

# To calculate bootstrap confidence intervals, you would typically sort the bootstrapped coefficients
# for each parameter and find the values at the desired percentiles (e.g., 2.5th and 97.5th for a 95% CI).
# Example (for the Intercept coefficient):
# intercept_ci_lower = boot_results_df['Intercept'].quantile(0.025)
# intercept_ci_upper = boot_results_df['Intercept'].quantile(0.975)

# Print the bootstrap estimated coefficients (mean).
print("\nBootstrap Model Coefficients (Mean):")
print(boot_means)

,sbp,sex,tox,grmhem,gestage,apgar5
0,43,Male,No,No,29,7
1,51,Male,No,No,31,8
2,42,Female,No,No,33,0
3,39,Female,No,No,31,8
4,48,Female,Yes,No,30,7


,sbp,sex,tox,grmhem,gestage,apgar5,gestage_low,apgar5_low
0,43,Male,No,No,29,7,0,0
1,51,Male,No,No,31,8,0,0
2,42,Female,No,No,33,0,0,1
3,39,Female,No,No,31,8,0,0
4,48,Female,Yes,No,30,7,0,0


Optimal Model Chosen: without interaction (sbp ~ C(gestage_low) + C(apgar5_low))

Optimal Model Coefficients (Theoretical):
Intercept              50.779943
C(gestage_low)[T.1]    -5.839828
C(apgar5_low)[T.1]     -3.264033
dtype: float64

Permutation Test p-values:
Gestage_low p-value: 0.0120
Apgar5_low p-value: 0.1760

Bootstrap Model Coefficients (Mean):
Intercept              50.778740
C(gestage_low)[T.1]    -5.970382
C(apgar5_low)[T.1]     -3.160825
dtype: float64
